# Data Cleaning & Preprocessing Notebook
This is where we will do any cleaning and standardizations to prepare for using ML models to predict DEATH_EVENT.  

*Note:* Some of these steps are redundant/unnecessary but are show for 'full lifecycle coverage'. Sometimes these two note books (ie. 01,02) are combined into 1 for simplicity.  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style of the plots
sns.set_style('darkgrid')

In [3]:
df = pd.read_csv("../data/heart_failure_clinical_records_dataset_cleaned.csv")

In [6]:
continuous_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

In [5]:
# Detect and handle missing values (already checked)
missing_values = df.isnull().sum()
print("Missing values:")
print(missing_values)

# Handle missing values - None


Missing values:
age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64


In [ ]:
# Remove Outliers (already checked)
quantile_1 = df[continuous_features].quantile(0.25)
quantile_3 = df[continuous_features].quantile(0.75)
iqr = quantile_3 - quantile_1
lower_bound = quantile_1 - 1.5 * iqr
upper_bound = quantile_3 + 1.5 * iqr

outliers = (df[continuous_features] < lower_bound) | (df[continuous_features] > upper_bound)

print("Outliers: ")
outliers.sum()
# Handle Outliers 
# Will skip this step as we did it in 01 EDA notebook 

Outliers: 


age                         0
creatinine_phosphokinase    0
ejection_fraction           0
platelets                   0
serum_creatinine            3
serum_sodium                0
time                        0
dtype: int64

### Standardization and Train Split 
Some of our ML models that we will be using require standardization of features. So we will first split our data, then standardize it.  

Process:  
- We will standardize only the continuous numerical features (age, creatinine_phosphokinase, ejection_fraction, platelets, serum_creatinine, serum_sodium, time) using StandardScaler.  

- Binary 0/1 variables were left unchanged. 

- Standardization was applied in a scikit-learn Pipeline, fit only on the training data, and then applied to the test set.   

- This was necessary for scale-sensitive models such as logistic regression, SVM, k-NN, and neural networks. 

- Tree-based models do not require scaling, but using a consistent preprocessing pipeline prevents data leakage and maintains comparability across models.



### Models That Require Feature Scaling

| **Model** | **Why It Needs Scaling** |
|----------|---------------------------|
| **Logistic Regression** | L2 regularization assumes all features are on the same scale; otherwise coefficients are penalized inconsistently. |
| **SVM (Linear, RBF kernels)** | Uses distance and dot products; features with larger magnitude dominate the decision boundary. |
| **k-NN** | Distance-based algorithm—unscaled features distort nearest-neighbor calculations. |
| **Neural Networks (PyTorch)** | Gradient-based optimization converges faster when inputs have similar scale; prevents exploding/vanishing gradients. |
| **PCA** | Computes variance along components; features with larger variance dominate unless scaled. |


In [ ]:
# Train, Validation, Test Split 